In [0]:
import torch
import torch.nn as nn
# 신경망 모델들이 포함되어있다.
import torch.optim as optim
#경사하강법 알고리즘이 포함되어있다.
import torch.nn.init as init
#텐서에 초기값을 주기위해 필요한 함수들이있다.
import matplotlib.pyplot as plt
import torchvision.datasets as dset
import torchvision.transforms as transforms
# torchvision은 유명한 영상처리용 데이터 셋, 모델, 이미지 변환기가 들어있는 패키지
# dataset 모듈은 데이터를 읽어오는 역할, transforms는 불러온 이미지를 필요에 따라 변환해주는 역할
from torch.utils.data import DataLoader
# 데이터를 하나씩 전달하지않고 원하는 배치사이즈대로 묶어서 전달하거나 더 효율적인 학습을 위해 데이터를
# 어떤 규칙에 따라 정렬하거나 섞어줄때 해주는 역할


In [0]:
batch_size = 256
learning_rate = 0.0002
num_epoch = 10


In [0]:
# 28x28 숫자 이미지로 이루어진 데이터셋 
mnist_train = dset.MNIST("./" , train=True , transform=transforms.ToTensor(),
                         target_transform = None , download = True)
# 두번째 인수는 훈련 여부를 나타냄 , 3,4 번째는 각각 데이터와 라벨에 대한 변형을 의미
# transform은 이미지에 대한 변형
mnist_test = dset.MNIST("./" , train=False , transform=transforms.ToTensor(),
                         target_transform = None , download = True)

train_loader = torch.utils.data.DataLoader(mnist_train,batch_size = batch_size , shuffle = True, num_workers= 2,
                                           drop_last= True)
#dset.MNIST를 통해 정리된 데이터를 batch_size 개수만큼 묶는다. 셔플 여부, 데이터를 묶을 떄 사용할 프로세스 개수, 묶고 남는 데이터는 버릴지 여부
test_loader = torch.utils.data.DataLoader(mnist_test,batch_size = batch_size , shuffle = False, num_workers= 2,
                                           drop_last= True)


0it [00:00, ?it/s]

9920512it [00:06, 1619845.42it/s]                             


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 137980.67it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw


1654784it [00:00, 2350545.47it/s]                           
0it [00:00, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


8192it [00:00, 53285.62it/s]            

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


In [0]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    #CNN클래스의 부모 클래스인 nn.Module을 초기화화는 역할을 한다.
    self.layer = nn.Sequential(
        nn.Conv2d(1,16,5),
        # 지금 MNIST 데이터셋은 [batch_size, 1, 가로(28) , 세로(28) ]로 구성되어있다. 
        # Conv2d는 여러개의 인수가 있는데 그중 in_channels, out_channels가 있다
        # 지금은 out_channels = 16, kernel_size= 5로 설정.
        nn.ReLU(),
        nn.Conv2d(16,32,5),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
        # MaxPool2d 의 인수는 kernel_size, stride , padding 등이 있다. kernel_size 는 풀링 연산할때 한번에 훑는 영역의 크기다.
        # k 를 전달하면 k x k 영역에서 풀링하고 stride 만큼 이동한다. 2x2 영역에서 풀링을 하고 2만큼 이동한다.
        nn.Conv2d(32,64,5),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
        # 출력 결과는 [batch_size, 64 ,3,3]
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(64*3*3,100),
        nn.ReLU(),
        nn.Linear(100,10)
        # 10개의 카테고리로 줄여주는 작업
    )

  def forward(self,x):
    out = self.layer(x)
    out = out.view(batch_size,-1)
    # view 함수에 인수로 목표하는 새로운 형태로 전달시켜준다.
    # [4,16] 이었으면 tensor.view(2,-1)를 거치면 [2,32]로 바꿔준다 여기서 -1 은 알아서 계산하라는 뜻.
    out = self.fc_layer(out)
    return out



In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [0]:
loss_arr = []
for i in range (num_epoch):
  for j,[image,label] in enumerate(train_loader):
    x = image.to(device)
    y_ = label.to(device)

    optimizer.zero_grad()
    output = model.forward(x)
    loss = loss_func(output,y_)
    loss.backward()
    optimizer.step()

    if j % 1000 == 0 :
      print(loss)
      loss_arr.append(loss.cpu().detach().numpy())

tensor(0.0574, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0260, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0530, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0445, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0171, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0532, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0338, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0152, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0209, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0068, device='cuda:0', grad_fn=<NllLossBackward>)


In [0]:
correct = 0
total = 0

with torch.no_grad():
  # 조건을 걸고 시작함. torch.no_grad(), 기울기를 계산하지 않겠다.

  for image,label in test_loader:
    x = image.to(device)
    y_ = label.to(device)

    output = model.forward(x)
    _,output_index = torch.max(output,1)
    # torch.max 함수를 사용해서 최대값과 인덱스를 구한다.
    total += label.size(0)
    correct += (output_index == y_).sum().float()

  print("Accuray of Test Data : {}".format(100*correct/total))

Accuray of Test Data : 98.9883804321289


In [0]:
# 컨볼루션 연산이 2번 연속하는 경우
# 컨볼루션-활성화함수-컨볼루션-활성화함수-풀링
def conv_2_block(in_dim,out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    return model

  
# 컨볼루션 연산이 3번 연속하는 경우
# 컨볼루션-활성화함수-컨볼루션-활성화함수-컨볼루션-활성화함수-풀링
def conv_3_block(in_dim,out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    return model

In [0]:
# 위에서 정의한 블록들을 이용해 VGG 네트워크를 만들어보겠습니다.
# 필터의 개수가 2의 n승의 값을 가지기 때문에 base_dim이란 변수를 추가해서 단순화 했습니다.
# 현재 dog, cat 두 가지 클래스를 구분하려고 하기 때문에 num_classes=2로 설정했습니다.

class VGG(nn.Module):
    def __init__(self, base_dim, num_classes=2):
        super(VGG, self).__init__()
        self.feature = nn.Sequential(
            conv_2_block(3,base_dim),
            conv_2_block(base_dim,2*base_dim),
            conv_3_block(2*base_dim,4*base_dim),
            conv_3_block(4*base_dim,8*base_dim),
            conv_3_block(8*base_dim,8*base_dim),            
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(8*base_dim * 7 * 7, 100),
            nn.ReLU(True),                                                      # True 는 inplace 연산을 하겠다는 의미를 가집니다. inplace 연산은 결과값을 새로운 변수에 값을 저장하는 대신 기존의 데이터를 대체하는것을 의미합니다.
            #nn.Dropout(),
            nn.Linear(100, 20),
            nn.ReLU(True),
            #nn.Dropout(),
            nn.Linear(20, num_classes),
        )

    def forward(self, x):
        x = self.feature(x)
        x = x.view(x.size(0), -1)                                               # x.size(0)를 batch size로 바꿔도 같은 값입니다.
        x = self.fc_layer(x)
        return x


In [0]:
# gpu가 사용 가능한 경우에는 device를 0번 gpu로 설정하고 불가능하면 cpu로 설정합니다.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# 앞서 정의한대로 vGG 클래스를 인스턴스화 하고 지정한 장치에 올립니다.
model = VGG(base_dim=16).to(device)

# 손실함수 및 최적화함수를 설정합니다.
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 모델 자녀 노드의 이름과 모듈을 출력합니다.
for i in model.named_children():
    print(i)

cuda:0
('feature', Sequential(
  (0): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (3): Seque

In [0]:
class VGGPro(nn.Module):
  def __init__(self,features,num_classes=1000, init_weights = True):
    super(VGGPro,self).__init()
    self.features = features
    self.avgpool = nn.AdaptiveAvgPool2d((7,7))
    self.classifier = nn.Sequential(
        nn.Linear(512 * 7 * 7 ,4096),
        nn.ReLU(),
        nn.Linear(4096,4096),
        nn.ReLU(),
        nn.Linear(4096, num_classes),
    )
    if init_weights:
      self._initialize_weights()

  def forward(self,x):
    # feature 에 들어가는 모델을 make_layers 로 직접 만들어준다.
    x = self.features(x)

    x = self.avgpool(x)
    x = x.view(x.size(0),-1)
    x = self.classifier(x)
    return x
  
  def _initialize_weights(self):
    for m in self.modules():
      if isinstance(m,nn.Conv2d):
        nn.init.kaiming_normal_(m.weight , mode= 'fan_out', nonlinearity='relu')
        if m.bias is not None:
          nn.init.constant_(m.bias,0)
      elif isinstance(m,nn.BatchNorm2d):
        nn.init.constant_(m.weight,1)
        nn.init.constant_(m.bias,0)
      elif isinstance(m,nn.Linear):
        nn.init.normal_(m.weight,0,0.01)
        nn.init.constant_(m.bias,0)

  def make_layers(cfg,batch_norm=False):
    layers = []
    # 빈 리스트 생
    in_channels = 3
    for v in cfg:
      if v == 'M':
        # 맥스풀링할 차례
        layers += [nn.MaxPool2d(kernel_size=2,stride=2)]
      else:
        # 맥스풀링 아니면 그 숫자만큼의 필터를 가진 합성곱 연산을 만들고 배치 정규화 여부에 따라 
        # [합성곱 연산 , 배치 정규화 , 렐루 활성화 함수], [합성곱 연산 , 렐루 활성화 함수]를 layer에 추
        conv2d = nn.Conv2d(in_channels , v, kernel_size=3 , padding=1)
        if batch_norm:
          layers += [conv2d , nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
        else:
          layers += [conv2d, nn.ReLU(inplace=True)]
          in_channels = v
    return nn.Sequential(*layers)
    # layer를 nn.Sequential로 감싸서 리턴해준다.
  
  cfg = {
      # 숫자는 필터를 의미, M 은 맥스풀링을 의미
      'A' : [64,'M',128,'M',256,256,'M',512,512,'M',512,512,'M'],

      'B' : [64,64,'M',128,126,'M',256,256,'M',512,512,'M',512,512,'M'],

      'D' : [64,64,'M',128,'M',256,256,256,'M',512,512,512,'M',512,512,512,'M'],

      'E' : [64,64,'M',128,128,'M',256,256,256,256,'M',512,512,512,512,'M',512,512,512,512,'M'],
  }